In [2]:
import gym
import numpy as np
import time
import random
import operator
import math
from IPython.display import clear_output
from gym import wrappers
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [3]:
env = gym.make("MountainCar-v0")
#env = wrappers.Monitor(env, '/tmp/cartpole-experiment-5')
env.reset();

In [4]:
env.step(1)

(array([-0.43690136, -0.00064762]), -1.0, False, {})

In [5]:
for i in range(100):
    env.step(random.randint(0,1))
    env.render()
env.reset()

array([-0.4084743,  0.       ])

In [6]:
pset = gp.PrimitiveSet("main", 2)
def if_then_else(input, output1, output2):
    return output1 if input else output2

def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1
    


pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(max, 2)
pset.addPrimitive(min, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)


In [7]:
expr = gp.genFull(pset, min_=1, max_=3)
tree = gp.PrimitiveTree(expr)

In [8]:
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

In [10]:
def sample_reward(env, func, t_max=200):
    s = env.reset()
    total_reward = 0
    max_pos = -10
    for _ in range(t_max):
        next_action = func(*s)
        if next_action<=-1:
            next_action = 0
        elif next_action>-1 and next_action<1:
            next_action = 1
        else:
            next_action = 2

        s, reward, is_done, _ = env.step(next_action)
        total_reward += reward
        if s[0]>max_pos:
            max_pos=s[0]
        if is_done:
            break
    return max_pos

In [14]:
def evalSymbReg(individual, n_times=10):
    
    func = toolbox.compile(expr=individual)
    rewards = [sample_reward(env, func) for _ in range(n_times)]
    
    return float(np.mean(rewards)), 

In [15]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=6)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [16]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)

In [1]:
pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)

NameError: name 'toolbox' is not defined

In [18]:
best_func = gp.compile(hof[0], pset)

In [19]:
evalSymbReg(hof[0])

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


(0.5300606692355054,)

In [ ]:
for i in range(100):
    s = env.reset()
    total_reward = 0
    for _ in range(500):
        next_action = best_func(*s)
        next_action = 0 if next_action<=0 else 1
        s, reward, is_done, _ = env.step(next_action)
        total_reward += reward
        env.render()
        if is_done:
            break


/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


TypeError: must be str, not bytes